# Worksheet 08

Name: William Huang
UID: U53888747

Github Repo Link (Branch worksheet_08): 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [98]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.892112516683211, 6.657883350732971, 4.475604685766358, 5.193203678122577, 5.787095309349438, 5.413155250523027, 4.294980883491939, 6.715436413386193, 5.078758840736582, 3.936652762100088]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [99]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.84020242270778, 8.07378021127034, 8.477711920654235, 6.116610290859272, 7.792637539287965, 7.706251428202184, 8.719879751432746, 8.775867944775392, 7.585194007796247, 8.546678903345612]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [100]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[3.936652762100088, 5.078758840736582, 6.715436413386193, 4.294980883491939, 5.413155250523027, 8.546678903345612, 7.585194007796247, 5.787095309349438, 8.775867944775392, 5.193203678122577]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean of s1: 5; Mean of s2: 8

Variance of s1: 1; Variance of s2: 1

Probability of a data point being in from s1 or s2: 0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [101]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.715436413386193, 8.546678903345612, 7.585194007796247, 8.775867944775392]
[3.936652762100088, 5.078758840736582, 4.294980883491939, 5.413155250523027, 5.787095309349438, 5.193203678122577]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 7.905794317325861,  mean_2 = 4.950641120720609
var_1 = 0.671874416930754,  var_2 = 0.4078147354129616


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [102]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x )
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  3.936652762100088
probability of observing that point if it came from cluster 0 =  3.940997128937347e-06
probability of observing that point if it came from cluster 1 =  0.17709747665532852
point =  5.078758840736582
probability of observing that point if it came from cluster 0 =  0.001271277349846033
probability of observing that point if it came from cluster 1 =  0.6122638832128457
point =  6.715436413386193
probability of observing that point if it came from cluster 0 =  0.16955576783617346
probability of observing that point if it came from cluster 1 =  0.013718726347984581
point =  4.294980883491939
probability of observing that point if it came from cluster 0 =  2.974697524009502e-05
probability of observing that point if it came from cluster 1 =  0.3687880741939148
point =  5.413155250523027
probability of observing that point if it came from cluster 0 =  0.004777135296811065
probability of observing that point if it came from cluster 1 =  0.4805891590689989
point =  8.

I think the estimated probablities look very correct to me -- the numbers closer to s_1 are marked as such and vice versa. There's a very clear separation between the two means, so it's very easy to spot which cluster a number came from, which lines up with the computer's guesses.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [103]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([(x - mean[0])**2 * p for x, p in zip(data, prob_s0_x)]) / sum(prob_s0_x), sum([(x - mean[1])**2 * p for x, p in zip(data, prob_s1_x)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 7.909342232030672,  mean_2 = 4.976000811073192
var_1 = 0.7077101192723146,  var_2 = 0.45442025110479417


They are pretty spot on... the values are incredibly similar.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [104]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x )
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([(x - mean[0])**2 * p for x, p in zip(data, prob_s0_x)]) / sum(prob_s0_x), sum([(x - mean[1])**2 * p for x, p in zip(data, prob_s1_x)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

point =  3.936652762100088
probability of observing that point if it came from cluster 0 =  6.815518074429961e-06
probability of observing that point if it came from cluster 1 =  0.18029366057910154
point =  5.078758840736582
probability of observing that point if it came from cluster 0 =  0.0016504111532805123
probability of observing that point if it came from cluster 1 =  0.5849724982260702
point =  6.715436413386193
probability of observing that point if it came from cluster 0 =  0.1732297091892115
probability of observing that point if it came from cluster 1 =  0.021201381306494464
point =  4.294980883491939
probability of observing that point if it came from cluster 0 =  4.6522760058118364e-05
probability of observing that point if it came from cluster 1 =  0.3552691423923665
point =  5.413155250523027
probability of observing that point if it came from cluster 0 =  0.005809454377844418
probability of observing that point if it came from cluster 1 =  0.47958005971995044
point =  

The values have very minimally changed, with a seeming shift of at most plus or minus 0.02 for each value.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [110]:
cluster_0 = []
cluster_1 = []

probs = zip(data, prob_s0_x, prob_s1_x)

for p in probs:
    if p[1] > p[2]:
        cluster_0.append(p[0])
    else:
        cluster_1.append(p[0])

print("Points from cluster 0: " + str(cluster_0))
print("Points from cluster 1: " + str(cluster_1))


Points from cluster 0: [6.715436413386193, 8.546678903345612, 7.585194007796247, 8.775867944775392]
Points from cluster 1: [3.936652762100088, 5.078758840736582, 4.294980883491939, 5.413155250523027, 5.787095309349438, 5.193203678122577]
